In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";"

)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
train,test = train_test_split(data,test_size=0.25,random_state=42)

In [4]:
train_x = train.drop(columns="quality").values
train_y = train["quality"].values.ravel() # ravel 2d array to 1d array

#test

test_x = test.drop(columns="quality").values
test_y = test["quality"].values.ravel()

# validation

train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature = infer_signature(train_x,train_y)

# ANN model

In [5]:
import mlflow.metrics
import mlflow.pytorch
import mlflow.tensorflow


def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    # nomalize
    mean = np.mean(train_x,axis=0)
    variance = np.var(train_x,axis=0)
    
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=variance),
            keras.layers.Dense(32,activation='relu'),
            keras.layers.Dense(1)
        ]
    )
    # compile the model
    model.compile(optimizer=keras.optimizers.SGD(
                 learning_rate=params["lr"],momentum=params["momentum"]
    ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    ## Train the model and track lr and momentum with MLFLOW
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=32)
        ## Evaluate
        eval_result = model.evaluate(valid_x,valid_y,batch_size=32)
        
        eval_rmse=eval_result[1]
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)
        # Log model
        mlflow.tensorflow.log_model(model,"model",signature=signature)
        
        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}

In [6]:
def objective(params):
    # Mlflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [7]:
space={
    "lr":hp.loguniform("lt",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [8]:
import mlflow.tensorflow


mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    trials = Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )
    best_run = sorted(trials.results,key=lambda x:x["loss"])[0]
    
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)
    
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 29s 328ms/step - loss: 29.3042 - root_mean_squared_error: 5.4133
43/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.5052 - root_mean_squared_error: 2.4050    
91/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.2296 - root_mean_squared_error: 1.9155
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.1766 - root_mean_squared_error: 1.9029 - val_loss: 0.6997 - val_root_mean_squared_error: 0.8365

Epoch 2/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.5162 - root_mean_squared_error: 0.7185
55/92 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - loss: 0.6740 - root_mean_squared_error: 0.8200
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6565 - root_mean_squared_error: 0.8096 - val_loss: 0.5875 - val_root_mean_squared_error: 0.7665

Epoch 3/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.7018 - root_mean_squared_error: 0.8377
45/

## Inferencing the model from MLFLOW

In [10]:
import mlflow
logged_model = 'runs:/ab4051f99a7a4c67b01e9e539350d6ff/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
data = test_x
# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[6.3757453],
       [6.4748273],
       [6.5147223],
       ...,
       [6.3015084],
       [6.8476996],
       [5.9480815]], dtype=float32)